In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Create a synthetic dataset
events = [
    'Address Change',
    'Pin Change',
    'Email Change',
    'Phone Number Change',
    'Username Change',
    'Password Reset',
    'Device Change',
    'Security Question Change',
    'Account Recovery',
    'Account Deactivation'
]

# Set the number of samples
num_samples = 10000

# Set the fraud probability
fraud_probability = 0.05

# Create the dataset
dataset = pd.DataFrame(columns=['Event', 'Fraud'])

for _ in range(num_samples):
    event = np.random.choice(events)
    fraud = np.random.choice([0, 1], p=[1-fraud_probability, fraud_probability])
    dataset = dataset.append({'Event': event, 'Fraud': fraud}, ignore_index=True)

# Save the dataset to a CSV file
dataset.to_csv('account_takeover_data.csv', index=False)


dataset.head()

C:\Users\pixel\AppData\Local\Temp\ipykernel_6116\1552522957.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = dataset.append({'Event': event, 'Fraud': fraud}, ignore_index=True)
C:\Users\pixel\AppData\Local\Temp\ipykernel_6116\1552522957.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = dataset.append({'Event': event, 'Fraud': fraud}, ignore_index=True)
C:\Users\pixel\AppData\Local\Temp\ipykernel_6116\1552522957.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = dataset.append({'Event': event, 'Fraud': fraud}, ignore_index=True)
C:\Users\pixel\AppData\Local\Temp\ipykernel_6116\1552522957.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a fut

,Event,Fraud
0,Password Reset,0
1,Phone Number Change,0
2,Security Question Change,0
3,Address Change,0
4,Address Change,0


In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences

# # Create a synthetic dataset
# events = [
#     'Address Change',
#     'Pin Change',
#     'Email Change',
#     'Phone Number Change',
#     'Username Change',
#     'Password Reset',
#     'Device Change',
#     'Security Question Change',
#     'Account Recovery',
#     'Account Deactivation'
# ]

# # Set the number of samples
# num_samples = 10000

# # Set the fraud probability
# fraud_probability = 0.05

# # Create the dataset
# dataset = pd.DataFrame(columns=['Event', 'Fraud'])

# for _ in range(num_samples):
#     event = np.random.choice(events)
#     fraud = np.random.choice([0, 1], p=[1-fraud_probability, fraud_probability])
#     dataset = dataset.append({'Event': event, 'Fraud': fraud}, ignore_index=True)

# # Save the dataset to a CSV file
# dataset.to_csv('account_takeover_data.csv', index=False)

# Load the dataset
dataset = pd.read_csv('account_takeover_data.csv')

# Preprocessing
label_encoder = LabelEncoder()
dataset['Event'] = label_encoder.fit_transform(dataset['Event'])

# Splitting the dataset into train and test sets
train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)

# Convert the sequential data into sequences of fixed length
sequence_length = 10  # Define the length of each sequence
num_features = len(dataset.columns) - 1

def create_sequences(data, labels):
    sequences = []
    for i in range(len(data) - sequence_length):
        sequence = data[i:i+sequence_length]
        sequences.append(sequence)
    return np.array(sequences), labels[:len(sequences)]

X_train, y_train = create_sequences(train_data.drop('Fraud', axis=1).values, y_train)
X_test, y_test = create_sequences(test_data.drop('Fraud', axis=1).values, y_test)


# Model architecture
model = Sequential()
model.add(LSTM(128, input_shape=(sequence_length, num_features), return_sequences=True))
model.add(LSTM(128))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, batch_size=32)
print(f'Loss: {loss}, Accuracy: {accuracy}')

# Make predictions
predictions = model.predict(X_test)


Epoch 1/10
250/250 [==============================] - 19s 23ms/step - loss: 0.2048 - accuracy: 0.9506
Epoch 2/10
250/250 [==============================] - 6s 23ms/step - loss: 0.1979 - accuracy: 0.9506
Epoch 3/10
250/250 [==============================] - 6s 23ms/step - loss: 0.1988 - accuracy: 0.9506
Epoch 4/10
250/250 [==============================] - 6s 23ms/step - loss: 0.1980 - accuracy: 0.9506
Epoch 5/10
250/250 [==============================] - 6s 23ms/step - loss: 0.1978 - accuracy: 0.9506
Epoch 6/10
250/250 [==============================] - 6s 23ms/step - loss: 0.1974 - accuracy: 0.9506
Epoch 7/10
250/250 [==============================] - 6s 22ms/step - loss: 0.1975 - accuracy: 0.9506
Epoch 8/10
250/250 [==============================] - 6s 23ms/step - loss: 0.1978 - accuracy: 0.9506
Epoch 9/10
250/250 [==============================] - 6s 23ms/step - loss: 0.1973 - accuracy: 0.9506
Epoch 10/10
63/63 [==============================] - 2s 9ms/step - loss: 0.1894 - accuracy

In [5]:
from sklearn.metrics import precision_score, recall_score

# Convert predictions to binary values
binary_predictions = (predictions > 0.5).astype(int)

# Calculate precision and recall
precision = precision_score(y_test, binary_predictions)
recall = recall_score(y_test, binary_predictions)

print(f'Precision: {precision}')
print(f'Recall: {recall}')


Precision: 0.0
Recall: 0.0


C:\Users\pixel\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In the line model.add(LSTM(128, input_shape=(sequence_length, num_features), return_sequences=True)), the number 128 refers to the number of LSTM units or memory cells in the LSTM layer.

LSTM (Long Short-Term Memory) is a type of recurrent neural network (RNN) that is well-suited for modeling sequential data. The LSTM layer consists of multiple memory cells that store and update information over time. Each memory cell has internal mechanisms to control the flow of information, allowing it to retain important information and forget irrelevant information.

The parameter 128 in this example is the number of LSTM units or memory cells in the layer. Choosing the number of units is a hyperparameter that you can adjust based on your specific problem and dataset. Increasing the number of units can potentially allow the model to capture more complex patterns in the sequential data, but it also increases the computational cost and may lead to overfitting if the dataset is small.

You can experiment with different values for the number of units, such as 64, 256, or other values, to find the optimal balance between model complexity and performance for your account takeover fraud detection task.

The line model.add(LSTM(128)) adds another LSTM layer to the model with 128 LSTM units.

In this case, the LSTM(128) layer is added without the return_sequences=True parameter. When return_sequences is not specified or set to False (default), the LSTM layer will only return the output of the last LSTM unit in the sequence, rather than returning the entire sequence of outputs. This is commonly used when the output of the LSTM layer is connected to a dense or output layer.

The number of LSTM units or memory cells, which is 128 in this example, determines the complexity and capacity of the LSTM layer. Each LSTM unit stores and updates information over time, and the number of units defines the dimensionality of the LSTM layer's internal representation space. More units can potentially capture more intricate patterns in the data, but it also increases model complexity and computational requirements.

By adding multiple LSTM layers to the model, the network can learn hierarchical representations of sequential patterns. Each layer can learn different levels of abstraction, with the initial layer capturing more basic patterns and the subsequent layers capturing more complex patterns.

It's important to note that the choice of the number of LSTM units is a hyperparameter that can be tuned to optimize model performance on your specific task and dataset. Different values, such as 64, 256, or others, can be experimented with to find the optimal balance between model complexity and predictive power.

In [8]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences


# Load the dataset
dataset = pd.read_csv('account_takeover_data.csv')

# Preprocessing
label_encoder = LabelEncoder()
dataset['Event'] = label_encoder.fit_transform(dataset['Event'])

# Splitting the dataset into train and test sets
train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)

# Convert the sequential data into sequences of fixed length
sequence_length = 10  # Define the length of each sequence
num_features = len(dataset.columns) - 1

def create_sequences(data, labels):
    sequences = []
    for i in range(len(data) - sequence_length):
        sequence = data[i:i+sequence_length]
        sequences.append(sequence)
    return np.array(sequences), labels[:len(sequences)]

X_train, y_train = create_sequences(train_data.drop('Fraud', axis=1).values, train_data['Fraud'].values)
X_test, y_test = create_sequences(test_data.drop('Fraud', axis=1).values, test_data['Fraud'].values)

# Model architecture
model = Sequential()
model.add(LSTM(128, input_shape=(sequence_length, num_features), return_sequences=True))
model.add(LSTM(64))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, batch_size=32)
print(f'Loss: {loss}, Accuracy: {accuracy}')

# Make predictions
predictions = model.predict(X_test)

# Convert predictions to binary values
binary_predictions = (predictions > 0.5).astype(int)

# Calculate precision and recall
from sklearn.metrics import precision_score, recall_score
precision = precision_score(y_test, binary_predictions)
recall = recall_score(y_test, binary_predictions)

print(f'Precision: {precision}')
print(f'Recall: {recall}')


Epoch 1/10
250/250 [==============================] - 11s 20ms/step - loss: 0.2120 - accuracy: 0.9476
Epoch 2/10
250/250 [==============================] - 5s 20ms/step - loss: 0.2067 - accuracy: 0.9476
Epoch 3/10
250/250 [==============================] - 5s 20ms/step - loss: 0.2068 - accuracy: 0.9476
Epoch 4/10
250/250 [==============================] - 5s 20ms/step - loss: 0.2073 - accuracy: 0.9476
Epoch 5/10
250/250 [==============================] - 5s 20ms/step - loss: 0.2067 - accuracy: 0.9476
Epoch 6/10
250/250 [==============================] - 5s 20ms/step - loss: 0.2065 - accuracy: 0.9476
Epoch 7/10
250/250 [==============================] - 5s 20ms/step - loss: 0.2064 - accuracy: 0.9476
Epoch 8/10
250/250 [==============================] - 5s 20ms/step - loss: 0.2067 - accuracy: 0.9476
Epoch 9/10
250/250 [==============================] - 5s 20ms/step - loss: 0.2072 - accuracy: 0.9476
Epoch 10/10
63/63 [==============================] - 2s 8ms/step - loss: 0.1974 - accuracy

C:\Users\pixel\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Finding complex sequential patterns

Finding complex sequential patterns often requires the use of advanced techniques and algorithms. Here are a few approaches you can consider:

Sequence Mining with Sequence Pattern Mining Algorithms: Sequence pattern mining algorithms, such as SPADE (Sequential PAttern Discovery using Equivalence classes) and PrefixSpan, can discover frequent sequential patterns in large datasets. These algorithms can handle complex patterns involving multiple events or transactions and can uncover hidden dependencies and associations.

Sequential Rule Mining: Sequential rule mining aims to discover rules that describe the temporal relationships between events or transactions. Algorithms like CMRules (Class-based Sequential Rule Mining) and GSP (Generalized Sequential Pattern) can extract complex rules that capture sequential patterns with various constraints, including time intervals, item gaps, and item repetitions.

Sequence Embeddings with Deep Learning: Deep learning techniques, such as sequence embeddings, can capture complex sequential patterns by mapping sequences into dense vector representations. Models like Sequence-to-Sequence (Seq2Seq) with attention mechanisms or Transformer-based models can learn rich representations of sequential data, enabling the discovery of intricate patterns. These embeddings can then be used for clustering, classification, or anomaly detection.

Graph-based Representations: Sequential data can often be represented as graphs, where events or transactions are nodes, and temporal relationships are edges. Graph mining techniques, such as graph-based sequence mining or graph neural networks, can capture complex patterns by leveraging the graph structure. These methods can model higher-order dependencies and capture patterns that span multiple sequences.

Temporal Convolutional Networks (TCNs): TCNs are deep learning models specifically designed for sequence analysis, capturing temporal dependencies at various scales. TCNs use dilated convolutions, which allow for exponentially increasing receptive fields, enabling the modeling of complex long-range patterns. They have been successful in capturing complex sequential patterns in various domains.

It's important to note that finding complex sequential patterns often involves a trade-off between the expressiveness of the model and the computational complexity. You may need to experiment with different approaches, algorithms, and parameter settings to strike the right balance for your specific problem and dataset. Additionally, feature engineering, preprocessing, and domain expertise play crucial roles in discovering and interpreting complex patterns in sequential data.

In [ ]:
Sequence Mining with Sequence Pattern Mining Algorithms (using PrefixSpan):

In [ ]:
from prefixspan import PrefixSpan

# Perform sequence mining using PrefixSpan
prefixspan = PrefixSpan(dataset)
frequent_patterns = prefixspan.frequent(minsup=2)


In [ ]:
Sequential Rule Mining (using CMRules):

In [ ]:
from cmrules import CMRules

# Perform sequential rule mining using CMRules
cmrules = CMRules(dataset)
rules = cmrules.mine(min_support=0.1, min_confidence=0.5)


In [ ]:
Sequence Embeddings with Deep Learning (using Seq2Seq with Attention):

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Attention

# Create a Seq2Seq model with Attention
model = Sequential()
model.add(LSTM(128, input_shape=(sequence_length, num_features), return_sequences=True))
model.add(Attention())
model.add(Dense(num_features, activation='softmax'))

# Compile and train the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=32)

# Generate sequence embeddings
sequence_embeddings = model.predict(X_test)


In [ ]:
Graph-based Representations (using NetworkX and graph mining algorithms):

In [ ]:
import networkx as nx
from stellargraph import StellarGraph
from stellargraph.data import BiasedRandomWalk
from stellargraph.data import UnsupervisedSampler
from stellargraph.mapper import GraphSAGENodeGenerator
from stellargraph.layer import GraphSAGE

# Convert sequential data to a graph representation
G = nx.Graph()
# Add nodes and edges to the graph based on your sequential data

# Create a StellarGraph object
graph = StellarGraph.from_networkx(G)

# Perform graph-based sequence mining
rw = BiasedRandomWalk(graph)
sequences = rw.run(nodes=list(graph.nodes()), length=10, n=100)
us = UnsupervisedSampler(graph, nodes=list(graph.nodes()), length=10, number_of_walks=100)
sequences = us.run()

# Use graph mining algorithms on the sequences
# Perform graph pattern mining or other graph-based analysis on the sequences


In [10]:
from itertools import combinations

# Convert your dataset to a list of sequences
sequences = [['A', 'B', 'C'], ['B', 'C', 'D'], ['A', 'C'], ['A', 'B', 'C', 'D'], ['B', 'D']]

# Set minimum support threshold
min_support = 2

# Step 1: Generate frequent 1-itemsets
frequent_1_itemsets = {}
for sequence in sequences:
    for item in sequence:
        if item in frequent_1_itemsets:
            frequent_1_itemsets[item] += 1
        else:
            frequent_1_itemsets[item] = 1

# Step 2: Generate frequent k-itemsets
k = 2
frequent_k_itemsets = {frozenset([item]): support for item, support in frequent_1_itemsets.items() if support >= min_support}
while frequent_k_itemsets:
    candidate_itemsets = set()
    for itemset1, itemset2 in combinations(frequent_k_itemsets, 2):
        candidate_itemsets.add(itemset1.union(itemset2))
    
    frequent_k_itemsets = {}
    for sequence in sequences:
        for candidate_itemset in candidate_itemsets:
            if candidate_itemset.issubset(sequence):
                if candidate_itemset in frequent_k_itemsets:
                    frequent_k_itemsets[candidate_itemset] += 1
                else:
                    frequent_k_itemsets[candidate_itemset] = 1
    
    frequent_k_itemsets = {itemset: support for itemset, support in frequent_k_itemsets.items() if support >= min_support}
    k += 1

# Extract frequent sequential patterns
frequent_patterns = [list(itemset) for itemset in frequent_k_itemsets]
frequent_patterns

[]

In [11]:
import pandas as pd

data = {
    'user_account': [1001, 1001, 1001, 1001, 1001, 1002, 1002, 1002, 1002, 1002, 1003, 1003, 1003, 1003, 1003],
    'sender': ['Alice', 'Alice', 'Bob', 'Alice', 'Bob', 'David', 'David', 'Eve', 'Eve', 'David', 'John', 'John', 'Mary', 'John', 'Mary'],
    'receiver': ['Bob', 'Bob', 'Alice', 'Bob', 'Charlie', 'Eve', 'Eve', 'David', 'David', 'Charlie', 'Mary', 'Mary', 'John', 'Mary', 'Charlie'],
    'transaction_amount': [100, 50, 75, 200, 150, 100, 50, 75, 200, 150, 500, 1000, 750, 2000, 1500],
    'transaction_type': ['payment', 'payment', 'transfer', 'payment', 'payment', 'payment', 'payment', 'transfer', 'payment', 'payment', 'payment', 'payment', 'transfer', 'payment', 'payment'],
    'label': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1]
}

df = pd.DataFrame(data)
df.head()

,user_account,sender,receiver,transaction_amount,transaction_type,label
0,1001,Alice,Bob,100,payment,0
1,1001,Alice,Bob,50,payment,0
2,1001,Bob,Alice,75,transfer,0
3,1001,Alice,Bob,200,payment,0
4,1001,Bob,Charlie,150,payment,0


In [14]:
import networkx as nx
import matplotlib.pyplot as plt

# Load the dataset
df = df

G = nx.from_pandas_edgelist(df, source='sender', target='receiver', create_using=nx.DiGraph)

# Plot the graph
pos = nx.spring_layout(G)
nx.draw(G, pos, with_labels=True)
plt.show()

TypeError: '_AxesStack' object is not callable

<Figure size 640x480 with 0 Axes>

In [ ]:
from sklearn.ensemble import IsolationForest

# Load the dataset
df = pd.read_csv('transactions.csv')

# Train an Isolation Forest model on legitimate transactions
legit_data = df[df['label'] == 0][['transaction_amount']]
model = IsolationForest(contamination=0.05)
model.fit(legit_data)

# Predict anomalies on new transactions
df['anomaly_score'] = model.decision_function(df[['transaction_amount']])
anomalies = df[df['anomaly_score'] < 0]


In [ ]:
Behavioral Profiling:

In [ ]:
import pandas as pd

# Load the dataset
df = pd.read_csv('transactions.csv')

# Calculate transaction statistics for each user account
user_stats = df.groupby('user_account').agg({
    'transaction_amount': ['mean', 'std'],
    'transaction_type': pd.Series.mode
}).reset_index()

# Identify unusual patterns based on changes in transaction behavior
unusual_patterns = user_stats[
    (user_stats['transaction_amount', 'std'] > 3 * user_stats['transaction_amount', 'mean']) |
    (user_stats['transaction_type', 'mode'] != 'payment')
]


In [ ]:
Time-based Analysis:

In [ ]:
import pandas as pd

# Load the dataset
df = pd.read_csv('transactions.csv')

# Convert the transaction timestamp to datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Sort the dataframe by user account and timestamp
df.sort_values(['user_account', 'timestamp'], inplace=True)

# Calculate the time duration between the first legitimate transaction and subsequent transactions
df['time_since_first_legit'] = df.groupby('user_account')['timestamp'].transform(lambda x: x - x.min())

# Identify suspicious patterns based on time duration
suspicious_patterns = df[df['time_since_first_legit'] > pd.Timedelta(days=30)]
